# implenenting KNN model for iris dataset
----

In [1]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
# knn
from sklearn.neighbors import KNeighborsClassifier
# linear models
from sklearn.linear_model import LogisticRegression
#knn and iris dataset
from knn import MyKNNClassifier
from sklearn.datasets import load_iris

### load iris dataset

In [2]:
data = load_iris()
df = pd.DataFrame(data=data.data, columns=data.feature_names)
df['target'] = data.target
df['target_name'] = df['target'].map(lambda i: data.target_names[i])

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


## Preprocessing

In [3]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Profiling Report")
profile.to_file("report.html")

ImportError: Numba needs NumPy 2.1 or less. Got NumPy 2.2.

In [4]:
y = df['target']
X = df.drop(['target','target_name'], axis=1)

random_state = 999
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)


In [5]:
NUMERICAL = list(X.columns)
CATEGORICAL = []

numerical_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, NUMERICAL)
    ]
)

knn_model = KNeighborsClassifier(n_neighbors=3)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', knn_model)
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['sepal length (cm)',
                                                   'sepal width (cm)',
                                                   'petal length (cm)',
                                                   'petal width (cm)'])])),
                ('classifier', KNeighborsClassifier(n_neighbors=3))])

In [6]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        12
           1     0.7143    0.8333    0.7692         6
           2     0.9091    0.8333    0.8696        12

    accuracy                         0.9000        30
   macro avg     0.8745    0.8889    0.8796        30
weighted avg     0.9065    0.9000    0.9017        30



In [25]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin


class MyKNNClassifier(BaseEstimator, ClassifierMixin):
    """
    Parameters
    ----------
    n_neighbors : int
        Number of neighbors (K).
    metric : str
        Distance metric to use. Supported: "euclidean", "manhattan".
    weighted : bool
        If True, weight neighbors by distance.
    """
    def __init__(self, n_neighbors=3, metric="euclidean", weighted=False):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.weighted = weighted

        # these will be set in fit()
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        self.classes_ = np.unique(y)
        return self
    
    def _compute_distances(self, X):
        if self.metric == "euclidean":
            return np.sqrt(((X[:, np.newaxis, :] - self.X_train[np.newaxis, :, :]) ** 2).sum(axis=2))
        elif self.metric == "manhattan":
            return np.abs(X[:, np.newaxis, :] - self.X_train[np.newaxis, :, :]).sum(axis=2)
        else:
            raise ValueError(f"Unsupported metric: {self.metric}")
        
        
    def _predict_one(self, distances_row):
        idx = np.argsort(distances_row)[:self.n_neighbors]
        neighbor_labels = self.y_train[idx]
        if self.weighted:
            neighbor_dist = distances_row[idx]
            weights = 1 / (neighbor_dist + 1e-9)
            unique_labels = np.unique(neighbor_labels)
            weighted_votes = [weights[neighbor_labels == lbl].sum() for lbl in unique_labels]
            return unique_labels[np.argmax(weighted_votes)]
        else:
            values, counts = np.unique(neighbor_labels, return_counts=True)
            return values[np.argmax(counts)]
        
        
    def predict(self, X):
        X = np.array(X)
        dists = self._compute_distances(X)
        return np.array([self._predict_one(row) for row in dists])


### Explanation: Implementation of `MyKNNClassifier`

This class implements a simple **K‑Nearest Neighbors (KNN)** classifier from scratch using **NumPy**, following the scikit‑learn API (with `fit()` and `predict()` methods).

---

#### 1. Overview

The KNN algorithm is a **non‑parametric** and **instance‑based** learning method.

For a test sample $\mathbf{x}_{test}$, it finds its $K$ nearest samples in the training set, 
and predicts the label $\hat{y}$ based on majority (or weighted) voting.

$$
\hat{y} = \arg\max_{c \in \mathcal{C}} \sum_{i \in \mathcal{N}_K(\mathbf{x}_{test})} w_i \, \mathbf{1}(y_i = c)
$$

Where:
- $\mathcal{N}_K(\mathbf{x}_{test})$: indices of $K$ nearest neighbors.
- $w_i = 1$ (unweighted) or $w_i = \frac{1}{d_i + \varepsilon}$ (weighted).
- $d_i$: distance between $\mathbf{x}_{test}$ and sample $\mathbf{x}_i$.

---

#### 2. The `fit()` Method
KNN does not learn parameters — it stores the training data:
$$
\text{Model memory} = { X_{\text{train}}, y_{\text{train}} }
$$

#### 3. Distance Computation
Distances are computed between all test samples and training samples in a vectorized NumPy form.
(a) Euclidean Distance
$$
d_{ij}^{(\text{euclidean})} = \sqrt{ \sum_{k=1}^{D} (x_{j,k}^{(test)} - x_{i,k}^{(train)})^2 }
$$

(b) Manhattan Distance
$$
d_{ij}^{(\text{manhattan})} = \sum_{k=1}^{D} |x_{j,k}^{(test)} - x_{i,k}^{(train)}|
$$

#### 4. Selecting Neighbors
we sort the distances and keep 𝐾 smallest ones:

$$
\mathcal{N}_K(\mathbf{x}) = \text{argsort}(d_i)[0:K]
$$

#### 5. Voting Strategy
(a) Unweighted Voting
Each neighbor contributes one vote.
$$
\hat{y} = \operatorname{mode}(y_i \mid i \in \mathcal{N}_K)
$$

(b) Weighted Voting
Closer neighbors have higher influence.
Weights depend inversely on distance:
$$
w_i = \frac{1}{d_i + 10^{-9}}
$$

Class vote score:

$$
S_c = \sum_{i \in \mathcal{N}_K(\mathbf{x}) ;:; y_i=c} w_i
$$

The predicted class is:
$$
\hat{y} = \arg\max_{c} S_c
$$

#### 6. The predict() Method
Computes full distance matrix and calls _predict_one() per test sample.
Each test sample votes independently using the above logic.

#### 7. Mathematical Summary
For each test sample $x \in \mathbb{R}^D$ :
Compute distances
$$
d_i = \begin{cases}\sqrt{\sum_k (x_k - x_{i,k})^2}, & \text{if metric = Euclidean} \\sum_k |x_k - x_{i,k}|, & \text{if metric = Manhattan}\end{cases}
$$

Find 𝐾 nearest neighbors:
$$
\mathcal{N}_K(\mathbf{x}) = \text{indices of 𝐾 smallest } d_i
$$

Vote by counts or weights
$$
S_c =
\begin{cases}\sum_{i} \mathbf{1}(y_i=c), & \text{if unweighted} \\sum_{i} \frac{\mathbf{1}(y_i=c)}{d_i+\varepsilon}, & \text{if weighted}\end{cases}
$$

Predict class
$$
\hat{y} = \arg\max_{c} S_c
$$

#### 8. Computational Complexity

Analyzing the computational cost of the **K‑Nearest Neighbors (KNN)** algorithm:

Distance Computation:

For each test sample, distances to all training samples are calculated.  
If we denote:

- $N_{\text{test}}$ → number of test samples  
- $N_{\text{train}}$ → number of training samples  
- $D$ → feature dimension  

then the time complexity of computing all pairwise distances is:

$$
O(N_{\text{test}} \times N_{\text{train}} \times D)
$$

Neighbor Sorting:

For each test instance, we need to sort the distances to find the $K$ smallest ones:

$$
O(N_{\text{train}} \log N_{\text{train}})
$$

per test sample.


Total Complexity:

Therefore, the total computational complexity of KNN prediction is approximately:

$$
O(N_{\text{test}} \times N_{\text{train}} \times D) \;+\;
O(N_{\text{test}} \times N_{\text{train}} \log N_{\text{train}})
$$

which can be simplified (dominant term) as:

$$
O(N_{\text{test}} \times N_{\text{train}} \times D)
$$

in practice, since distance computation usually dominates sorting for small $K$.


In [26]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

preprocessor = ColumnTransformer([
    ('num', MinMaxScaler(), list(X.columns))
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MyKNNClassifier(n_neighbors=5, metric='euclidean', weighted=True))
])

# metric_choice = 'manhattan' 
# weighted_mode = True   

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        12
           1     0.7143    0.8333    0.7692         6
           2     0.9091    0.8333    0.8696        12

    accuracy                         0.9000        30
   macro avg     0.8745    0.8889    0.8796        30
weighted avg     0.9065    0.9000    0.9017        30



$$
\textbf{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$  
**Meaning:** The overall proportion of correct predictions (both positive and negative) among all samples.

---

$$
\textbf{Precision} = \frac{TP}{TP + FP}
$$  
**Meaning:** Of all the samples predicted as positive, how many are actually positive.

---

$$
\textbf{Recall} = \frac{TP}{TP + FN}
$$  
**Meaning:** Of all the actual positive samples, how many were correctly identified by the model.

---

$$
\textbf{F1\text{-}Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$  
**Meaning:** The harmonic mean of Precision and Recall — gives a balanced measure of accuracy when the dataset is imbalanced.


In [28]:
from sklearn.model_selection import GridSearchCV

# Define the pipeline again (if needed)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MyKNNClassifier())
])

# Define parameter grid to test all combinations
param_grid = {
    'classifier__n_neighbors': [1, 3, 5, 7, 9, 11],
    'classifier__metric': ['euclidean', 'manhattan'],
    'classifier__weighted': [True, False]
}

# Grid search with 5-fold cross validation
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

# Fit search on the entire training set
grid_search.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate on the held-out test set
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

from sklearn.metrics import classification_report, accuracy_score
print("\nFinal Test Accuracy:", accuracy_score(y_test, y_pred_best))

Best Parameters: {'classifier__metric': 'euclidean', 'classifier__n_neighbors': 5, 'classifier__weighted': False}
Best Cross-Validation Accuracy: 0.9833333333333334

Final Test Accuracy: 0.9


### Searching for the Optimal KNN Configuration

To optimize the performance of our custom `MyKNNClassifier`, 
we systematically explore different hyperparameter configurations.

#### Parameters Examined
| Parameter | Description | Tested Values |
|------------|--------------|---------------|
| `n_neighbors (K)` | Number of nearest neighbors | 1, 3, 5, 7, 9, 11 |
| `metric` | Distance metric used | `euclidean`, `manhattan` |
| `weighted` | Voting scheme | `True` (weighted) or `False` (unweighted) |

The search procedure uses **Grid Search with 5‑fold Cross‑Validation** 
to find the combination producing the highest average accuracy.

Mathematically, the optimization process solves:

$$
(\hat{K}, \hat{m}, \hat{w}) =
\arg\max_{K, m, w}
 \frac{1}{5} \sum_{i=1}^5 
 \text{Accuracy}_i(K, m, w)
$$

where:
- $K$: number of neighbors  
- $m \in \{\text{Euclidean}, \text{Manhattan}\}$  
- $w \in \{\text{weighted}, \text{unweighted}\}$  

---

#### Final Reporting

At the end, report:

. **Best parameter combination**